## Import Statements

In [ ]:
import pandas as pd
import numpy as np
import requests
import spacy
import os
import tensorflow as tf

import tensorflow as tf
from tensorflow import keras
from sklearn.pipeline import Pipeline
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.optimizers import SGD, Adam, Adagrad, Nadam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, ReLU
from tensorflow.keras.activations import sigmoid, softmax, relu
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from spacy.tokenizer import Tokenizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import re, string, timeit
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV

# import gensim
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel
# import pyLDAvis
# import pyLDAvis.gensim 
import matplotlib.pyplot as plt

## Import data

In [ ]:
PATH = '/content/listings_summary.csv'

In [ ]:
listings = pd.read_csv(PATH, index_col='id')

In [ ]:
listings.head()

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,...,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,"No parties No events No pets No smoking, not e...",NaN,NaN,https://a0.muscache.com/im/pictures/260fd609-7...,NaN,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",Believe in sharing economy.,within an hour,96%,NaN,t,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,Mitte,4.0,4.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,Mitte,...,2.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",NaN,$60.00,NaN,NaN,$200.00,$30.00,1,$28.00,4,1125,3 months ago,t,0,21,51,141,2018-11-07,118,2016-04-11,2018-10-28,93.0,10.0,9.0,10.0,10.0,10.0,9.0,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76
2695,https://www.airbnb.com/rooms/2695,20181107122246,2018-11-07,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,none,NaN,NaN,Within walking distance you'll find the S-Bahn...,Außer deinem Zimmer kannst du noch die Küche u...,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/97d4f6e8-1...,NaN,2986,https://www.airbnb.com/users/show/2986,Michael,2008-09-16,"Berlin, Berlin, Germany",Living with my wife in Berlin. Like travellin...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/50434955-1...,https://a0.muscache.com/im/pictures/50434955-1...,Prenzlauer Berg,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Berlin, Berlin, Germany",NaN,Prenzlauer Berg Nordwest,Pankow,...,1.0,Real Bed,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",NaN,$17.00,NaN,NaN,$0.00,$0.00,1,$0.00,2,40,7 weeks ago,t,0,0,0,0,2018-11-07,6,2018-07-04,2018-10-01,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,NaN,f,f,flexible,f,f,1,1.42
3176,https://www.airbnb.com/rooms/3176,20181107122246,2018-11-07,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,The neighbourhood is famous for its variety of...,We welcome FAMILIES and cater especially for y...,"We 

In [ ]:
listings.columns

Index(['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url',
       'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedr

## Feature Engineering

In [ ]:
'''
INDEX:
id

property_type
beds
bathrooms
amenities
price
minimum_nights
review_score_rating
neighbourhood_group_cleansed

BINARY:
**host_identity_verified
cleaning_fee
security_deposit

'''

'\nINDEX:\nid\n\nproperty_type\nbeds\nbathrooms\namenities\nprice\nminimum_nights\nreview_score_rating\nneighbourhood_group_cleansed\n\nBINARY:\n**host_identity_verified\ncleaning_fee\nsecurity_deposit\n\n'

In [ ]:
cols = ['beds', 'property_type', 'price', 'description', 
                'minimum_nights', 'review_scores_rating',
                 'neighbourhood_group_cleansed',
                 'host_identity_verified', 'cleaning_fee',
                 'security_deposit']


listings = listings[cols]

In [ ]:
listings.head(1)

,beds,property_type,price,description,minimum_nights,review_scores_rating,neighbourhood_group_cleansed,host_identity_verified,cleaning_fee,security_deposit
id,,,,,,,,,,
2015,2.0,Guesthouse,$60.00,Great location! 30 of 75 sq meters. This wood...,4,93.0,Mitte,t,$30.00,$200.00


In [ ]:
cols = ['cleaning_fee', 'security_deposit']

In [ ]:
# remove '$' and ',' so that we can process a monteary amount 
# as a float, versus an object

In [ ]:
listings[cols] = listings[cols].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [ ]:
listings['price'] = listings['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [ ]:
listings['cleaning_fee']

id
2015        30.0
2695         0.0
3176        50.0
3309        30.0
7071         0.0
            ... 
29856708     NaN
29857108     0.0
29864272     NaN
29866805     NaN
29867352     NaN
Name: cleaning_fee, Length: 22552, dtype: float64

In [ ]:
# create binary column for cleaning fee or not

In [ ]:
def binary_fee(x):
  if x > 0.0:
    return "Yes"
  else:
    return "No"

In [ ]:
listings['cleaning_fee_binary'] = listings['cleaning_fee'].apply(binary_fee)

In [ ]:
listings['cleaning_fee_binary'].value_counts()

Yes    13541
No      9011
Name: cleaning_fee_binary, dtype: int64

In [ ]:
listings['security_deposit_binary'] = listings['security_deposit'].apply(binary_fee)

In [ ]:
listings['security_deposit_binary'].value_counts()

No     14680
Yes     7872
Name: security_deposit_binary, dtype: int64

In [ ]:
# dropping monetary columns because it will overwhelm
# our computer later on
drop_cols = ['security_deposit', 'cleaning_fee']

listings = listings.drop(drop_cols, axis=1)

In [ ]:
listings.head(2)

,beds,property_type,price,description,minimum_nights,review_scores_rating,neighbourhood_group_cleansed,host_identity_verified,cleaning_fee_binary,security_deposit_binary
id,,,,,,,,,,
2015,2.0,Guesthouse,60.0,Great location! 30 of 75 sq meters. This wood...,4,93.0,Mitte,t,Yes,Yes
2695,1.0,Apartment,17.0,In the summertime we are spending most of our ...,2,100.0,Pankow,t,No,No


## NLP and text analysis

In [ ]:
listings['description'] = listings['description'].astype(str)

In [ ]:
listings['property_type'] = listings['property_type'].astype(str)

In [ ]:
# downloading the language libraries
# %%capture
# !python -m spacy download en_core_web_md 
# !python -m spacy download de_core_news_md
# !python -m spacy download xx_ent_wiki_sm

In [ ]:
# defining each language library
# English
# nlp_en = spacy.load('en_core_web_md')
# German
# nlp_de = spacy.load('de_core_news_md')
# Multi
# nlp_multi = spacy.load('xx_ent_wiki_sm')

### Tfidf Vectorizer (if we wanted a non-NN model to do text analysis)

In [ ]:
# def tokenize(text):
    
#   lemmas = []  
      
#   doc = nlp_en(text)

#   for token in doc:
#     if((token.is_stop == False ) & (token.is_punct == False)):
#         lemmas.append(token.lemma_.lower())

#   return lemmas

In [ ]:
# def proccess_text(text):
#   lang = detect_lang(text)
#   sent_langs = []
#   doc = nlp_en(text)
#   for sent in doc.sents:
#     lang_pred = sent._.language['language']
#     lang = 'de' if lang_pred in ['de', 'nl'] else 'en'
#     sent_langs.append((sent.text, lang))
#   processed_text = []
#   for sent in sent_langs:
#     if sent[1] == 'de':
#       tokens = [token.lemma_ for token in nlp_de(sent[0])]
#     else:
#       tokens = [token.lemma_ for token in nlp_en(sent[0])]
#     for token in tokens:
#       processed_text.append(token)
#   return processed_text

In [ ]:
# listings['lemmas'] = listings['description'].apply(tokenize)

In [ ]:
# listings['amenities'][2]

In [ ]:
# vects = [nlp_en(string).vector for string in listings['description']]

In [ ]:
# listings['vects'] = vects 

In [ ]:
# pipe = Pipeline([('tfidf', TfidfVectorizer()), ('clf', KNeighborsClassifier())])

# pipe.fit(listings['description'], listings['price'])

In [ ]:
# parameters = {
#     'tfidf__max_df':(0.25, 0.75, 1),
#     'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    
# }

# grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
# grid_search.fit(listings['description'], listings['price'])

In [ ]:
# grid_search.best_score_

In [ ]:
# listings['lemmas'][2]

In [ ]:
# test = """
# My modern 2-bedroom apartment has everything you need for your Berlin trip. 
# The unit comes with heating, a washer, and free parking.
#  During your stay, you can also enjoy using a convenient gym and kitchen. 
# Our Airbnb is within walking distance to several popular restaurants and shops. 
# An ideal base to explore Berlin.

# """

In [ ]:
# grid_search.predict([test])

## Data Splitting

In [ ]:
variety_threshold = 40 # Anything that occurs less than this will be removed.
value_counts = listings['property_type'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
listings.replace(to_remove, np.nan, inplace=True)
listings = listings[pd.notnull(listings['property_type'])]

In [ ]:
train_size = int(len(listings) * .8)

# Train features
description_train = listings['description'][:train_size]
variety_train = listings['property_type'][:train_size]

# Train labels
labels_train = listings['price'][:train_size]

# Test features
description_test = listings['description'][train_size:]
variety_test = listings['property_type'][train_size:]

# Test labels
labels_test = listings['price'][train_size:]

### Baseline Score

In [ ]:
print("Description Baseline Score: ", description_train.value_counts(normalize=True).max())

Description Baseline Score:  0.0006129841181387574


## Compiled-Model Neural Network

In [ ]:
layers = keras.layers

# This code was tested with TensorFlow v1.7
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.3.0


In [ ]:
listings['property_type'].value_counts()

Apartment             20225
Condominium             612
Loft                    460
House                   398
Serviced apartment      175
Hostel                  128
Townhouse                99
Guest suite              74
Bed and breakfast        64
Guesthouse               57
Hotel                    50
Other                    47
Boutique hotel           43
Name: property_type, dtype: int64

In [ ]:
vocab_size = 6000
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train)

In [ ]:
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [ ]:
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_classes = np.max(variety_train) + 1

# Convert labels to one hot
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

In [ ]:
# Hyperparameter tuning
logdir = os.path.join("logs", "EarlyStopping-loss")
tb_callback = TensorBoard(logdir, histogram_freq=5)
stop = EarlyStopping(monitor='loss',
                     min_delta=0.01845,
                     patience=2)

#### Wide Model

In [ ]:
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
merged_layer = layers.Dense(896, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [ ]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6000)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 6013)         0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          1539584     concatenate[0][0]     

In [ ]:
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

### Deep Model

In [ ]:
deep_inputs = layers.Input(shape=(max_seq_length))
# input layer
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)

# hidden layers
embedding = layers.Flatten()(embedding)
# model = tf.expand_dims(embed_out, axis=-1)
# embed_out = layers.Dense(10, activation="relu")(embedding)
embed_out = layers.Dense(256, activation="relu")(embedding)
embed_out = layers.Dense(896, activation="relu")(embedding)
embed_out = layers.Dense(1)(embedding)

# output layer
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 170)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 170, 8)            48000     
_________________________________________________________________
flatten (Flatten)            (None, 1360)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1361      
Total params: 49,361
Trainable params: 49,361
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

### Combined Model

In [ ]:
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(10, activation="relu")(merged_out)
merged_out = layers.Dense(256, activation="relu")(merged_out)
merged_out = layers.Dense(896)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6000)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 6013)         0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 170)]        0                                 

In [ ]:
combined_model.fit([description_bow_train, variety_train] + [train_embed], 
                   labels_train, epochs=20, batch_size=40, 
                   verbose=2, callbacks=[stop])

Epoch 1/20
449/449 - 2s - loss: 15640.4922 - accuracy: 1.6718e-04
Epoch 2/20
449/449 - 2s - loss: 12923.2686 - accuracy: 1.6718e-04
Epoch 3/20
449/449 - 2s - loss: 8318.6162 - accuracy: 2.7863e-04
Epoch 4/20
449/449 - 2s - loss: 5770.9575 - accuracy: 3.9008e-04
Epoch 5/20
449/449 - 2s - loss: 4872.2705 - accuracy: 5.5726e-04
Epoch 6/20
449/449 - 2s - loss: 3246.9482 - accuracy: 1.6718e-04
Epoch 7/20
449/449 - 2s - loss: 5533.6890 - accuracy: 1.1145e-04
Epoch 8/20
449/449 - 2s - loss: 2008.5858 - accuracy: 5.5726e-05
Epoch 9/20
449/449 - 2s - loss: 1337.1089 - accuracy: 0.0028
Epoch 10/20
449/449 - 2s - loss: 1717.3251 - accuracy: 0.0017
Epoch 11/20
449/449 - 2s - loss: 1846.8669 - accuracy: 9.4734e-04


### Evaluating the Combined Model and Making Predictions

In [ ]:
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], 
                        labels_test, batch_size=40, verbose=2)

113/113 - 0s - loss: 176553.8594 - accuracy: 0.0000e+00


[176553.859375, 0.0]

In [ ]:
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

In [ ]:
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

Eine schöne und gemütliche privat Wohnung. Ideal für Alleinerziehende mit 3 Kindern.  Zentral in Berlin. Gute Anbindung an das Nahverkehr. Netter Kiez.
Predicted:  39.013763 Actual:  30.0 

Ich studiere Architektur und beschäftige mich mit der Nachhaltigkeit. Die Wohnung liegt im 1OG ( alt bau )  Und da ich Architektur studiere und mich mit der Nachhaltigkeit beschäftige sind alle Möbel die Ihr vorfinden werdet alle aus Recycelt quelle was  der Wohnung seinen speziellen Charme verleiht. KIEZ live
Predicted:  75.924484 Actual:  55.0 

Die Wohnung befindet sich im ersten Stock eines neunstöckigen Hochhauses. Vor der Tür ist leicht, einen kostenfreien Parkplatz zu finden. Der Bus fährt keine 50m von dem Haus ab und in 10 Minuten erreicht er den Alexanderplatz. Einkaufsmöglichkeiten gibt es auch in der Nähe. Die Gegend ist sehr ruhig aber dank der guten Anbindung, ist man sehr schnell mitten drin, wo das Berliner Leben pulsiert.
Predicted:  33.41594 Actual:  30.0 

One room in a equipped a

In [ ]:
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  17.80019521713257


## Pickling Model

In [ ]:
import pickle
from pickle import dump

In [ ]:
from keras.models import load_model

combined_model.save('best_model.h5')

## Miscellanous Code from Experimentation Stage

In [ ]:
####################

In [ ]:
# train_size = int(len(listings) * .8)

# # Train features
# description_train = listings['description'][:train_size]
# # variety_train = listings['test_amenities'][:train_size]

# # Train labels
# labels_train = listings['price'][:train_size]

# # Test features
# description_test = listings['description'][train_size:]
# # variety_test = listings['test_amenities'][train_size:]

# # Test labels
# labels_test = listings['price'][train_size:]

In [ ]:
# vocab_size = 20000
# tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
# tokenize.fit_on_texts(description_train)

In [ ]:
# description_bow_train = tokenize.texts_to_matrix(description_train)
# description_bow_test = tokenize.texts_to_matrix(description_test)

In [ ]:
# # Hyperparameter tuning
# logdir = os.path.join("logs", "EarlyStopping-loss")
# tb_callback = TensorBoard(logdir, histogram_freq=3)
# stop = EarlyStopping(monitor='loss',
#                      min_delta=0.0184,
#                      patience=2)

In [ ]:
# train_embed = tokenize.texts_to_sequences(description_train)
# test_embed = tokenize.texts_to_sequences(description_test)

# max_seq_length = 250
# train_embed = keras.preprocessing.sequence.pad_sequences(
#     train_embed, maxlen=max_seq_length, padding="post")
# test_embed = keras.preprocessing.sequence.pad_sequences(
#     test_embed, maxlen=max_seq_length, padding="post")

In [ ]:
# deep_inputs = layers.Input(shape=(max_seq_length))
# # input layer
# embedding = layers.Embedding(vocab_size, 15, input_length=max_seq_length)(deep_inputs)

# # hidden layers
# embedding = layers.Flatten()(embedding)
# embed_out = layers.Dense(1)(embedding)
# # model = tf.expand_dims(embed_out, axis=-1)
# embed_out = layers.LSTM(32, activation="relu")
# embed_out = layers.Dense(52, activation="relu")(embedding)
# embed_out = layers.Dense(256, activation="relu")(embedding)
# # embed_out = layers.Dense(256, activation="relu")(embedding)


# # output layer
# embed_out = layers.Dense(896, activation="relu")(embedding)
# deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
# print(deep_model.summary())

In [ ]:
# deep_model.compile(loss='mse',
#                        optimizer='adam',
#                        metrics=['accuracy'])

In [ ]:
# deep_model.fit(train_embed,
#               labels_train, epochs=20, batch_size=32,
#                callbacks=[tb_callback, stop],
#                verbose=2)

In [ ]:
# deep_model.evaluate(test_embed, 
#                     labels_test, 
#                     batch_size=250, verbose=2)

In [ ]:
# predictions1 = deep_model.predict([test_embed])

In [ ]:
# num_predictions = 40
# diff = 0

# for i in range(num_predictions):
#     val = predictions1[i]
#     print(description_test.iloc[i])
#     print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
#     diff += abs(val[0] - labels_test.iloc[i])

In [ ]:
###############################

In [ ]:
# merged_out = layers.concatenate([wide_model.output, deep_model.output])
# merged_out = layers.Dense(1)(merged_out)
# combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
# print(combined_model.summary())

# combined_model.compile(loss='mse',
#                        optimizer='nadam',
#                        metrics=['accuracy'])

In [ ]:
# combined_model.fit([description_bow_train, variety_train] + [train_embed], 
#                    labels_train, epochs=20, batch_size=32, 
#                     callbacks=[tb_callback, stop], verbose=2)

In [ ]:
# deep_model.evaluate([description_bow_test, variety_test] + [test_embed], 
#                         labels_test, batch_size=128, verbose=2)

In [ ]:
# predictions = deep_model.predict([description_bow_test, variety_test] + [test_embed])

In [ ]:
# num_predictions = 40
# diff = 0

# for i in range(num_predictions):
#     val = predictions[i]
#     print(description_test.iloc[i])
#     print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
#     diff += abs(val[0] - labels_test.iloc[i])

In [ ]:
# from tensorflow.keras.models import Sequential

# from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Adagrad, Adamax, Ftrl, Nadam
# from tensorflow.keras.layers import Dense
# import tensorflow.keras.backend as K
# import tensorflow as tf

In [ ]:
# def customize_relu(alpha):
#   return lambda x: tf.keras.activations.relu(x, alpha=alpha)

In [ ]:
# def create_model(hp):


#   model = Sequential()

#   hp_units = hp.Int('units', 32, 512, 32)
  
#   alphas  = hp.Float('relu_alphas', 0.0, 0.1, 0.02)
  
#   for i in range(hp.Int('num_layers', 2, 20)):
#     model.add(Dense(units=hp_units,
#                     activation=customize_relu(alphas)))
    
#   lrs = hp.Choice('learning_rate',
#                    values=[.001, .01, .1]
#                   )
#   optimizer_choices = hp.Choice('optimizer',
#                                 ['Adadelta', 'Adagrad',
#                                   'Adamax', 'Ftrl', 'Nadam'])
   
#   optimizer = eval(f'{optimizer_choices}(learning_rate={lrs})')
  

#   model.add(Dense(10, activation='softmax'))


#   # model compiling
#   model.compile(optimizer=optimizer,
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])

#   return model

In [ ]:
# !pip install keras-tuner

In [ ]:
# from tensorflow import keras
# from tensorflow.keras import layers
# from kerastuner.tuners import RandomSearch

In [ ]:
# !pip install category_encoders
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import seaborn as sns
# from sklearn.impute import SimpleImputer
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline
# from sklearn.ensemble import RandomForestClassifier
# from category_encoders import OneHotEncoder, OrdinalEncoder
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_selection import SelectKBest
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier

In [ ]:
# X_train = listings.drop(['price', 'description', 'amenities'], axis=1)[:train_size]

In [ ]:
# y_train = listings['price'][:train_size]

In [ ]:
# X_test = listings.drop(['price', 'description', 'amenities'], axis=1)[train_size:]
# y_test = listings['price'][train_size:]

In [ ]:
# model = make_pipeline(
#     OneHotEncoder(use_cat_names=True),
#     SimpleImputer(),
#     DecisionTreeClassifier(random_state=42)
# )

# model.fit(X_train, y_train);

In [ ]:
# training_acc = model.score(X_train, y_train)
# validation_acc = model.score(X_test, y_test)
# print('Training Accuracy Score:', training_acc)
# print('Validation Accuracy Score:', validation_acc)

In [ ]:
# tuner = RandomSearch(
#     create_model,
#     objective='val_accuracy',
#     max_trials=5,
#     executions_per_trial=3,
#     directory='./',
#     project_name='final')

In [ ]:
# tuner.search_space_summary()

In [ ]:
# tuner.search(X_train, y_train,
#              epochs=5,
#              validation_data=(X_test, y_test))